### install & download

In [1]:
# !pip install HanTa

In [ ]:
# !pip install transformers

In [3]:
# !pip3 install wordfreq

In [4]:
# !python -m spacy download de_core_news_md

In [5]:
# !python -m spacy download ru_core_news_md

In [6]:
# !wget https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt

In [7]:
# !pip install torch

### import & load

https://github.com/aalok-sathe/surprisal
does not support bert

In [67]:
import attr

import numpy as np
import networkx as nx
import nltk
import nltk.stem
import re
import spacy
import torch

from collections import Counter
from functools import cached_property, cache
from HanTa import HanoverTagger as ht
from nltk.corpus import stopwords
from spacy.lang.de.examples import sentences
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertForNextSentencePrediction
from typing import Callable, Dict, Optional, List
from wordfreq import word_frequency

In [9]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/galina.ryazanskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/galina.ryazanskaya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/galina.ryazanskaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
nlp_de = spacy.load("de_core_news_md")
nlp_ru = spacy.load("ru_core_news_md")

In [12]:
stopwords_ru = stopwords.words("russian")

In [13]:
stopwords_de = stopwords.words("german")

In [ ]:
# german_stopwords = ["a","ab","aber","ach","acht","achte","achten","achter","achtes","ag","alle","allein","allem","allen","aller","allerdings","alles","allgemeinen","als","also","am","an","ander","andere","anderem","anderen","anderer","anderes","anderm","andern","anderr","anders","au","auch","auf","aus","ausser","ausserdem","außer","außerdem","b","bald","bei","beide","beiden","beim","beispiel","bekannt","bereits","besonders","besser","besten","bin","bis","bisher","bist","c","d","d.h","da","dabei","dadurch","dafür","dagegen","daher","dahin","dahinter","damals","damit","danach","daneben","dank","dann","daran","darauf","daraus","darf","darfst","darin","darum","darunter","darüber","das","dasein","daselbst","dass","dasselbe","davon","davor","dazu","dazwischen","daß","dein","deine","deinem","deinen","deiner","deines","dem","dementsprechend","demgegenüber","demgemäss","demgemäß","demselben","demzufolge","den","denen","denn","denselben","der","deren","derer","derjenige","derjenigen","dermassen","dermaßen","derselbe","derselben","des","deshalb","desselben","dessen","deswegen","dich","die","diejenige","diejenigen","dies","diese","dieselbe","dieselben","diesem","diesen","dieser","dieses","dir","doch","dort","drei","drin","dritte","dritten","dritter","drittes","du","durch","durchaus","durfte","durften","dürfen","dürft","e","eben","ebenso","ehrlich","ei","ei,","eigen","eigene","eigenen","eigener","eigenes","ein","einander","eine","einem","einen","einer","eines","einig","einige","einigem","einigen","einiger","einiges","einmal","eins","elf","en","ende","endlich","entweder","er","ernst","erst","erste","ersten","erster","erstes","es","etwa","etwas","euch","euer","eure","eurem","euren","eurer","eures","f","folgende","früher","fünf","fünfte","fünften","fünfter","fünftes","für","g","gab","ganz","ganze","ganzen","ganzer","ganzes","gar","gedurft","gegen","gegenüber","gehabt","gehen","geht","gekannt","gekonnt","gemacht","gemocht","gemusst","genug","gerade","gern","gesagt","geschweige","gewesen","gewollt","geworden","gibt","ging","gleich","gott","gross","grosse","grossen","grosser","grosses","groß","große","großen","großer","großes","gut","gute","guter","gutes","h","hab","habe","haben","habt","hast","hat","hatte","hatten","hattest","hattet","heisst","her","heute","hier","hin","hinter","hoch","hätte","hätten","i","ich","ihm","ihn","ihnen","ihr","ihre","ihrem","ihren","ihrer","ihres","im","immer","in","indem","infolgedessen","ins","irgend","ist","j","ja","jahr","jahre","jahren","je","jede","jedem","jeden","jeder","jedermann","jedermanns","jedes","jedoch","jemand","jemandem","jemanden","jene","jenem","jenen","jener","jenes","jetzt","k","kam","kann","kannst","kaum","kein","keine","keinem","keinen","keiner","keines","kleine","kleinen","kleiner","kleines","kommen","kommt","konnte","konnten","kurz","können","könnt","könnte","l","lang","lange","leicht","leide","lieber","los","m","machen","macht","machte","mag","magst","mahn","mal","man","manche","manchem","manchen","mancher","manches","mann","mehr","mein","meine","meinem","meinen","meiner","meines","mensch","menschen","mich","mir","mit","mittel","mochte","mochten","morgen","muss","musst","musste","mussten","muß","mußt","möchte","mögen","möglich","mögt","müssen","müsst","müßt","n","na","nach","nachdem","nahm","natürlich","neben","nein","neue","neuen","neun","neunte","neunten","neunter","neuntes","nicht","nichts","nie","niemand","niemandem","niemanden","noch","nun","nur","o","ob","oben","oder","offen","oft","ohne","ordnung","p","q","r","recht","rechte","rechten","rechter","rechtes","richtig","rund","s","sa","sache","sagt","sagte","sah","satt","schlecht","schluss","schon","sechs","sechste","sechsten","sechster","sechstes","sehr","sei","seid","seien","sein","seine","seinem","seinen","seiner","seines","seit","seitdem","selbst","sich","sie","sieben","siebente","siebenten","siebenter","siebentes","sind","so","solang","solche","solchem","solchen","solcher","solches","soll","sollen","sollst","sollt","sollte","sollten","sondern","sonst","soweit","sowie","später","startseite","statt","steht","suche","t","tag","tage","tagen","tat","teil","tel","tritt","trotzdem","tun","u","uhr","um","und","uns","unse","unsem","unsen","unser","unsere","unserer","unses","unter","v","vergangenen","viel","viele","vielem","vielen","vielleicht","vier","vierte","vierten","vierter","viertes","vom","von","vor","w","wahr","wann","war","waren","warst","wart","warum","was","weg","wegen","weil","weit","weiter","weitere","weiteren","weiteres","welche","welchem","welchen","welcher","welches","wem","wen","wenig","wenige","weniger","weniges","wenigstens","wenn","wer","werde","werden","werdet","weshalb","wessen","wie","wieder","wieso","will","willst","wir","wird","wirklich","wirst","wissen","wo","woher","wohin","wohl","wollen","wollt","wollte","wollten","worden","wurde","wurden","während","währenddem","währenddessen","wäre","würde","würden","x","y","z","z.b","zehn","zehnte","zehnten","zehnter","zehntes","zeit","zu","zuerst","zugleich","zum","zunächst","zur","zurück","zusammen","zwanzig","zwar","zwei","zweite","zweiten","zweiter","zweites","zwischen","zwölf","über","überhaupt","übrigens"]

In [14]:
flatten = lambda l: [item for sublist in l for item in sublist]

## GloVe German

In [15]:
punct = '\!"#$%&\'\(\)\*\+,\-\./:;<=>\?@[\\]^_`{|}~\^„“/…'
def test_unicode(text):
    non_unicode = re.findall(f'[^A-Za-zÀ-ž0-9\\n {punct}]', text)
    return not bool(non_unicode), non_unicode

In [ ]:
with open('vectors.txt') as f:
    lines = f.readlines()

In [ ]:
def process_line(line):
    key, str_value = line.strip().split(' ', 1)
    return key, np.array([float(x) for x in str_value.split()])

In [ ]:
glove = {}
for line in lines:
    key, value = process_line(line)
    if test_unicode(key):
        glove[key] = value

In [ ]:
glove_oov = set()
def get_glove_vector(word):
    if word in glove:
        return glove[word]
    else:
        glove_oov.add(word)
        return None

#### Save glove

In [ ]:
import gensim
from tqdm import tqdm

In [ ]:

def save_word2vec_format(fname, vocab, vector_size, binary=True):
  # gensim 3.4
    """Store the input-hidden weight matrix in the same format used by the original
    C word2vec-tool, for compatibility.

    Parameters
    ----------
    fname : str
        The file path used to save the vectors in.
    vocab : dict
        The vocabulary of words.
    vector_size : int
        The number of dimensions of word vectors.
    binary : bool, optional
        If True, the data wil be saved in binary word2vec format, else it will be saved in plain text.


    """

    total_vec = len(vocab)
    with gensim.utils.open(fname, 'wb') as fout:
        print(total_vec, vector_size)
        fout.write(gensim.utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
        # store in sorted order: most frequent words at the top
        for word, row in tqdm(vocab.items()):
            if binary:
                row = row.astype(np.float32)
                fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
            else:
                fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))

In [ ]:
%%time
# save_word2vec_format(binary=True, fname='/content/drive/MyDrive/studies/science/Schizophrenia_private/german/glove_de.bin', vocab=glove, vector_size=300)

In [ ]:
# model = gensim.models.KeyedVectors.load_word2vec_format('glove_de.bin', binary=True)

## Preprocess
https://textmining.wp.hs-hannover.de/Preprocessing.html

https://spacy.io/usage/linguistic-features

In [ ]:
#from pymystem3 import Mystem
#mystem = Mystem()
# def preprocess_text_ru(text):
#     tokens = mystem.lemmatize(text.lower())
#     tokens = [token for token in tokens if token not in stopwords_ru\
#               and token != " " \
#               and token.strip() not in punct]

#     text = " ".join(tokens)

#     return text

In [16]:
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [ ]:
default_sent_tokenizer = lambda t: nltk.sent_tokenize(t, 'german')
default_word_tokenizer = lambda t: nltk.word_tokenize(t, 'german')
# default_stemmer = lambda w: nltk.stem.snowball.GermanStemmer().stem(w)
default_pos_tagger = lambda w: tagger.analyze(w)[-1]
default_lemmatizer = lambda w: tagger.analyze(w)[0]
default_model = lambda w: get_glove_vector(w)

In [17]:
def idf_sent_vectors(words: List[str], vectors: List[np.array]) -> np.array:
    assert len(words) == len(vectors)
    weights = [word_frequency(w, 'de') for w in words]
    return np.average(vectors, axis=0, weights=weights)

In [18]:
@attr.s(auto_attribs=True)
class TextData():
    text: str
    words: Optional[List[str]] = attr.ib(factory=list)
    lemmas: Optional[List[str]] = attr.ib(factory=list)
    pos: Optional[List[str]] = attr.ib(factory=list)
    sents: Optional[List[str]] = attr.ib(factory=list)
    sent_words: Optional[List[List[str]]] = attr.ib(factory=list)
    word_vectors: Optional[List[np.array]] = attr.ib(factory=list)
    word_for_vectors: Optional[List[str]] = attr.ib(factory=list)
    sent_word_vectors: Optional[List[List[np.array]]] = attr.ib(factory=list)
    sent_word_for_vectors: Optional[List[List[str]]] = attr.ib(factory=list)
    oov: Optional[List[str]] = attr.ib(factory=list)

    def __attrs_post_init__(self):
        self.sent_vectors = [idf_sent_vectors(words, vectors) for words, vectors in zip(self.sent_word_for_vectors, self.sent_word_vectors)]

In [ ]:
def process_default(text: str, stopwords: Optional[List[str]] = []) -> TextData:
    words = default_word_tokenizer(text)
    sents = default_sent_tokenizer(text)
    sent_words = [default_word_tokenizer(sent) for sent in sents]
    pos = [default_pos_tagger(w) for w in words]
    lemmas = [default_lemmatizer(w) for w in words]
    word_vectors = [default_model(w.lower()) for w in lemmas if default_model(w.lower()) is not None]
    word_for_vectors = [w for w in lemmas if default_model(w.lower()) is not None]
    oov = [w for w in lemmas if default_model(w.lower()) is None]
    sent_word_vectors = [[default_model(default_lemmatizer(w).lower()) \
                          for w in sent if \
                          default_model(default_lemmatizer(w).lower()) is not None]
                         for sent in sent_words]
    sent_word_for_vectors = [[default_lemmatizer(w) for w in sent if \
                              default_model(default_lemmatizer(w).lower()) \
                              is not None]
                              for sent in sent_words]
    data = TextData(text=text, words=words, lemmas=lemmas, pos=pos, sents=sents,
                    sent_words=sent_words, word_vectors=word_vectors, oov=oov,
                    sent_word_vectors=sent_word_vectors,
                    word_for_vectors=word_for_vectors,
                    sent_word_for_vectors=sent_word_for_vectors)
    return data

In [19]:
def process_spacy(text: str, stopwords: Optional[List[str]] = [], nlp=nlp_de) -> TextData:
    doc = nlp(text)
    words = [token for token in doc if token.text not in stopwords]
    sents = [str(sent) for sent in doc.sents]
    sent_words = [[token for token in nlp(sent) if token.text not in stopwords]
                   for sent in sents]
    pos = [token.pos_ for token in doc]
    lemmas = [token.lemma_ for token in doc]
    filter_token = lambda token: token.text not in stopwords and not token.is_oov and token.pos_ not in ['PUNCT', 'NUM']
    cleaned_sent_words = [[t for t in s if filter_token(t)] for s in sent_words]
    cleaned_sent_words = [s for s in cleaned_sent_words if s]
    oov = list(set(token.text for token in doc
                   if (token.text not in stopwords
                       and token.is_oov
                       and token.pos_ not in ['PUNCT', 'NUM'])))
    sent_word_vectors = [[token.vector for token in s] for s in cleaned_sent_words]
    sent_word_for_vectors = [[token.text for token in s] for s in cleaned_sent_words]
    word_vectors = flatten(sent_word_vectors)
    word_for_vectors = flatten(sent_word_for_vectors)
    data = TextData(text=text, words=words, lemmas=lemmas, pos=pos, sents=sents,
                sent_words=sent_words, word_vectors=word_vectors, oov=oov,
                sent_word_vectors=sent_word_vectors,
                word_for_vectors=word_for_vectors,
                sent_word_for_vectors=sent_word_for_vectors)
    return data

In [69]:
some_large_text = "Der Dackel zeichnet sich durch niedrige, kurzläufige, langgestreckte, aber kompakte Gestalt aus. Er ist sehr muskulös, mit aufrechter Haltung des Kopfes und aufmerksamem Gesichtsausdruck. Die verkürzten Beine der Dackel sind das Resultat einer gezielten Selektion auf Chondrodysplasie und sind im Rassestandard verankert. Es gibt die Hunde in verschiedenen Größen und Fellvariationen: Langhaar, Rauhaar, Kurzhaar in jeweils vielen Farbvariationen, auch mehrfarbig, gestromt oder gefleckt. Während gefleckte Tiere als eine der 4 Färbungen durch den FCI definiert sind, werden Schwarze Tiere ohne Brand sowie weiße mit oder ohne Brand im Rassestandard der FCI ausdrücklich ausgeschlossen[2]. Unter AKC-Regeln können weiß gescheckte Dackel ausgestellt werden und sind dort als Piebalds bekannt.[3] Die hängenden Ohren sind nicht zu weit vorne angesetzt, ausreichend, aber nicht übertrieben lang und abgerundet. In den drei Haararten werden die Dackel im FCI-Standard nach ihrer Größe unterschieden in Teckel (T) (früher Normalteckel), Brustumfang (BU) über 35 cm, Gewichtsobergrenze etwa 9 kg, Zwergteckel (Zw), BU über 30 bis 35 cm, und Kaninchenteckel (Kt), BU bis 30 cm."

In [ ]:
# data_default = process_default(some_large_text)

In [70]:
data = process_spacy(some_large_text)

In [88]:
type(nlp_de)

spacy.lang.de.German

In [ ]:
some_large_text_ru = "Знаете ли вы? Будапештский университет иудаики, около 1890 года Социалистическая власть не мешала работе раввинской семинарии (на илл.). Будапештский университет иудаики был открыт в 1877 году, через несколько десятилетий после того, как в Падуе, Меце, Париже и Бреслау были построены первые европейские раввинские семинарии. Тем не менее, он остаётся старейшим существующим учебным заведением в мире, где готовят раввинов."

In [ ]:
data_ru = process_spacy(some_large_text_ru, nlp=nlp_ru)

## Lexical

In [ ]:
def TTR(words: List[str]) -> float:
    return len(set(words)) / len(words)

In [ ]:
TTR(data.lemmas)

In [ ]:
def MATTR(words: List[str], w: Optional[int] = 10) -> float:
    if len(words) <= w:
        return TTR(words)
    else:
        ttrs = []
        for i in range(len(words)-w):
            ttrs.append(TTR(words[i:i+w]))
        return np.mean(ttrs)

In [ ]:
MATTR(data.lemmas)

## Syntactic

In [ ]:
def pos_rates(pos: List[str]) -> Dict[str, float]:
    return {k: v/len(pos) for k, v in Counter(pos).items()}

In [ ]:
pos_rates(data.pos)

In [ ]:
def sent_lengts(sents_words: List[List[str]]) -> float:
    return np.mean([len(sent) for sent in sents_words])

In [ ]:
sent_lengts(data.sent_words)

## Graph

In [ ]:
class _graphStatistics():

	def __init__(self, graph: nx.MultiDiGraph):
		self.graph = graph

	def statistics(self) -> Dict[str, float]:
		res = {}
		graph = self.graph
		res['number_of_nodes'] = graph.number_of_nodes()
		res['number_of_edges'] = graph.number_of_edges()
		res['PE'] =  (np.array(list(Counter(graph.edges()).values()))>1).sum()
		res['LCC'] = len(max(nx.weakly_connected_components(graph), key=len))
		res['LSC'] = len(max(nx.strongly_connected_components(graph), key=len))

		degrees = [v for k, v in graph.degree()]
		res['degree_average'] =  np.mean(degrees)
		res['degree_std'] =  np.std(degrees)


		adj_matrix = nx.adjacency_matrix(graph).toarray()
		adj_matrix2 = np.dot(adj_matrix , adj_matrix)
		adj_matrix3 = np.dot(adj_matrix2 , adj_matrix)

		res['L1'] =  np.trace(adj_matrix)
		res['L2'] =  np.trace(adj_matrix2)
		res['L3'] =  np.trace(adj_matrix3)

		return res


class naiveGraph():
	def __init__(self):
		pass

	def _text2graph(self, words: List[str]) -> nx.MultiDiGraph:
		gr = nx.MultiDiGraph()
		gr.add_edges_from(zip(words[:-1], words[1:]))
		return gr

	def analyzeText(self, words: List[str]) -> Dict[str, float]:
		dgr = self._text2graph(words)
		return _graphStatistics(dgr).statistics()

# https://github.com/facuzeta/speechgraph/blob/master/speechgraph/speechgraph.py

In [ ]:
naiveGraph().analyzeText(data.lemmas)

In [ ]:
def moving_graph_statistics(words: List[str], w: Optional[int] = 10) -> Dict[str, float]:
    if len(words) <= w:
        return naiveGraph().analyzeText(words)
    else:
        graph_stats = {}
        for i in range(len(words)-w):
            current_stats = naiveGraph().analyzeText(words[i:i+w])
            graph_stats = {key: graph_stats.get(key, []) + [value] for key, value in current_stats.items()}
        return {key: np.mean(value) for key, value in graph_stats.items()}

In [ ]:
moving_graph_statistics(data.lemmas, 15)

## LM

In [ ]:
def cos_sim(v1, v2):
    return np.inner(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [ ]:
def get_local_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    if len(clause_vectors) <= 1:
        return [np.nan]
    local_coherence_list = []
    for i in range(len(clause_vectors)-1):
        local_coherence_list.append(cos_sim(clause_vectors[i], clause_vectors[i+1]))
    return local_coherence_list

In [ ]:
def get_second_order_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    if len(clause_vectors) <= 1:
        return [np.nan]
    SOC_list = []
    for i in range(len(clause_vectors)-2):
        SOC_list.append(cos_sim(clause_vectors[i], clause_vectors[i+2]))
    return SOC_list

In [ ]:
def compare_text_to_a_standard_vector(clause_vectors: List[np.array], standard_vector: np.array) -> float:
    average_file_vector = np.average(clause_vectors, axis=0)
    return cos_sim(average_file_vector, standard_vector)

In [ ]:
def get_global_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    if len(clause_vectors) <= 1:
        return [np.nan]
    standard_vector = np.average(clause_vectors, axis=0)
    return [cos_sim(vec, standard_vector) for vec in clause_vectors]

In [ ]:
def get_cumulative_global_coherence_list(clause_vectors: List[np.array]) -> List[float]:
    if len(clause_vectors) <= 1:
        return [np.nan]
    cumulative_global_coherence_list = []
    for i in range(len(clause_vectors)):
        cumulative_vector = np.average(clause_vectors[:i+1], axis=0)
        cumulative_global_coherence_list.append(cos_sim(clause_vectors[i], cumulative_vector))
    return cumulative_global_coherence_list

### BERT

https://aclanthology.org/2020.coling-main.598/

In [ ]:
model_name_de = "bert-base-german-cased"
model_de = AutoModel.from_pretrained(model_name_de)
tokenizer_de = AutoTokenizer.from_pretrained(model_name_de)

In [ ]:
model_name_ru = "DeepPavlov/rubert-base-cased"
model_ru = AutoModel.from_pretrained(model_name_ru)
tokenizer_ru = AutoTokenizer.from_pretrained(model_name_ru)

### sent vectorize

In [ ]:
inputs = tokenizer_de(some_large_text, return_tensors="pt")
outputs, outputs_1 = model_de(**inputs).to_tuple()

In [ ]:
inputs = tokenizer_ru(some_large_text_ru, return_tensors="pt")
outputs, outputs_1 = model_ru(**inputs).to_tuple()

In [ ]:
def bert_sent_vectorize(text: str, model=model_de, tokenizer=tokenizer_de) -> np.array:
    inputs = tokenizer(text, return_tensors="pt")
    word_vectors, sent_vector = model(**inputs).to_tuple()
    return sent_vector.detach().squeeze().numpy()

In [ ]:
bert_sent_vectorize(data.sents[0])[:10]

In [ ]:
bert_sent_vectorize(data_ru.sents[0], model=model_ru, tokenizer=tokenizer_ru)[:10]

### next sentence prediction

In [ ]:
tokenizer_nsp_de = BertTokenizer.from_pretrained(model_name_de)
model_nsp_de = BertForNextSentencePrediction.from_pretrained(model_name_de)
tokenizer_nsp_ru = BertTokenizer.from_pretrained(model_name_ru)
model_nsp_ru = BertForNextSentencePrediction.from_pretrained(model_name_ru)

In [ ]:
def next_sent_prob(sent_text_1: str, sent_text_2: str,
                   tokenizer_nsp=tokenizer_nsp_de,
                   model_nsp=model_nsp_de) -> float:
    tokenized = tokenizer_nsp(sent_text_1, sent_text_2, return_tensors='pt')
    predict = model_nsp(**tokenized)
    pred = torch.nn.functional.softmax(predict.logits[0], dim=0)[0].item()
    return pred

In [ ]:
sentence_A = "Ich bin nicht so sicher."
sentence_B = "Jedoch nur die Rasse Dachshund enthält."

In [ ]:
next_sent_prob(sentence_A, sentence_B)

In [ ]:
sentence_A_ru = "Ну что такое?"
sentence_B_ru = "Почему-то все опять не работает."

In [ ]:
next_sent_prob(sentence_A_ru, sentence_B_ru, tokenizer_nsp=tokenizer_nsp_ru, model_nsp=model_nsp_ru)

In [ ]:
def get_prob_list(sents: List[str], tokenizer_nsp=tokenizer_nsp_de, model_nsp=model_nsp_de) -> List[float]:
    return [next_sent_prob(sents[i], sents[i+1], tokenizer_nsp=tokenizer_nsp, model_nsp=model_nsp) for i in range(len(sents)-1)]

In [ ]:
get_prob_list(data.sents)

In [ ]:
get_prob_list(data_ru.sents, tokenizer_nsp=tokenizer_nsp_ru, model_nsp=model_nsp_ru)

## Apply

In [ ]:
@attr.s(auto_attribs=True)
class ProcessTextData:
    data: TextData
    bert_lang: Optional[str] = attr.ib(default='de')

    def __attrs_post_init__(self):
        if self.bert_lang == 'de':
            self.bert_model = model_de
            self.bert_model_nsp = model_nsp_de
            self.bert_tokenizer = tokenizer_de
            self.bert_tokenizer_nsp = tokenizer_nsp_de
        elif self.bert_lang == 'ru':
            self.bert_model = model_ru
            self.bert_model_nsp = model_nsp_ru
            self.bert_tokenizer = tokenizer_ru
            self.bert_tokenizer_nsp = tokenizer_nsp_ru

        self.n_words = len(self.data.words)
        self.LTR = self.TTR()
        self.MALTR = self.MATTR()
        self.lexical = {'n_words': self.n_words,
                        'LTR': self.LTR,
                        'MALTR': self.MALTR}

        self.n_sents = len(self.data.sents)
        self.mean_sent_words = np.mean([len(s) for s in self.data.sent_words])
        self.pos_rates = pos_rates(self.data.pos)

        self.syntactic = {'n_sents': self.n_sents,
                        'mean_sent_words': self.mean_sent_words}
        self.syntactic.update(self.pos_rates)

        self.graph_statistics = self.graph_statistics()

        self.mean_lcoh = np.mean(self.local_coherence_list())
        self.mean_gcoh = np.mean(self.global_coherence_list())
        self.mean_cgcoh = np.mean(self.cumulative_global_coherence_list())
        self.mean_scoh = np.mean(self.second_order_coherence_list())
        self.mean_sent_prob = np.mean(self.sent_prob_list)

        self.LM = {'m_lcoh': self.mean_lcoh,
                   'm_gcoh': self.mean_gcoh,
                   'm_cgcoh': self.mean_cgcoh,
                   'm_scoh': self.mean_scoh,
                   'm_sporb': self.mean_sent_prob,
                   'm_bert_lcoh': np.mean(self.local_coherence_list(model='bert')),
                   'm_bert_gcoh': np.mean(self.global_coherence_list(model='bert')),
                   'm_bert_cgcoh': np.mean(self.cumulative_global_coherence_list(model='bert')),
                   'm_bert_scoh': np.mean(self.second_order_coherence_list(model='bert'))
                   }


    # lexical
    def TTR(self, lemmas: bool = True) -> float:
        items = self.data.lemmas if lemmas else self.data.words
        return TTR(items)

    def MATTR(self, w: Optional[int] = 10, lemmas: bool = True) -> float:
        items = self.data.lemmas if lemmas else self.data.words
        return MATTR(items, w=w)

    # graph
    def graph_statistics(self, w: Optional[int] = 10, lemmas: bool = True) -> Dict[str, float]:
        items = self.data.lemmas if lemmas else self.data.words
        return moving_graph_statistics(items, w=w)

    # LM
    @cached_property
    def bert_sent_vectors(self) -> List[np.array]:
        return [bert_sent_vectorize(s,
                                    model=self.bert_model,
                                    tokenizer=self.bert_tokenizer)
                for s in self.data.sents]

    def sent_vectors(self, model: Optional[str] = 'default') -> List[np.array]:
        if model == 'bert':
            return self.bert_sent_vectors
        else:
            return self.data.sent_vectors

    def local_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
        return get_local_coherence_list(self.sent_vectors(model=model))

    def global_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
        return get_global_coherence_list(self.sent_vectors(model=model))

    def cumulative_global_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
        return get_cumulative_global_coherence_list(self.sent_vectors(model=model))

    def second_order_coherence_list(self, model: Optional[str] = 'default') -> List[float]:
        return get_second_order_coherence_list(self.sent_vectors(model=model))

    @cached_property
    def sent_prob_list(self):
        return get_prob_list(self.data.sents,
                             model_nsp=self.bert_model_nsp,
                             tokenizer_nsp=self.bert_tokenizer_nsp)

    # values
    @cached_property
    def values(self):
        return {'LM': self.LM, 'syntactic': self.syntactic,
                'lexical': self.lexical, 'graph': self.graph_statistics}

In [ ]:
d = ProcessTextData(data)
d.values

In [54]:
word_frequency('работе', 'ru')

0.000182

In [ ]:
d_ru = ProcessTextData(data_ru, bert_lang='ru')
d_ru.values

In [ ]:
def average_values(list_of_values_dicts: List[Dict[str, Dict[str, float]]]) -> Dict[str, Dict[str, float]]:
    new_val = {metric_type: {metric_name: [] for metric_name in metric_dict.keys()}
               for metric_type, metric_dict in list_of_values_dicts[0].items()}
    for values in list_of_values_dicts:
        for metric_type, d in values.items():
            for metric_name, metric_value in d.items():
                if metric_name not in new_val[metric_type]:
                    new_val[metric_type][metric_name] = [metric_value]
                new_val[metric_type][metric_name].append(metric_value)

    return {metric_type: {metric_name: np.mean(metric_values) for metric_name,
                        metric_values in metric_dict.items()}
             for metric_type, metric_dict in new_val.items()}

Test

In [41]:
import pandas as pd
df = pd.read_csv('split_questions_cp_0.tsv', sep='\t', index_col=0).dropna()

In [ ]:
newrows = {}
for index, row in df.iterrows():
    vals = []
#     print(len(row))
    for w in row:
        d = process_spacy(w)
        v = ProcessTextData(d).values
        vals.append(v)
        del d
    newrows[index] = average_values(vals)
#     print(row['anger'])

In [65]:
for i, x in df['preprocessed_transcript'].items():
    print(i, x)

45_6M_manual_c  Ja was bedeutet Traurigkeit. Also generell habe ich halt ein Problem mit Emotionen und ähm. Also ich bin eigentlich eher ein rational orientierter Mensch. Aber jetzt so in letzter Zeit lerne ich das halt so kennen. Weil ich ja eben diese verschiedenen Wechsel tagtäglich erlebe. Ja ja Trauer ist halt Trauer. Was soll ich jetzt dazu sagen?  Mhm ja. Also das ist halt wenn ich. Also ich kann das halt eben bei einem anderen Menschen also fühlen. Wenn in einem anderen Menschen sehr viel Trauer drinne ist und ich mit dem verbunden bin dann nehme ich diese Trauer wahr. Ja. Ja ansonsten ist es halt so dass ich meine eigenen Emotionen ja eigentlich mein Leben lang unterdrückt habe und ja. Ähm. Es ist halt dann. Es gibt halt dann so Situationen wo Trauer da ist. Und wo es dann zum Weinen kommt und ich dadurch dann die Trauer auflöse. Und es danach dann besser wird. Hatte ich naja. Also da jetzt gestern vorgestern war jetzt einiges da so mit Frauen und sowas halt. Und in Bezug auf 

In [ ]:
for_pd = []
for i, d in newrows.items():
    d_p = {'index' : i}
    for dd in d.values():
        d_p.update(dd)
    for_pd.append(d_p)
new_df = pd.DataFrame(for_pd)

In [ ]:
new_df.head()

In [ ]:
with open('NET_NAP_new_.tsv', 'w') as f:
    f.write(new_df.to_csv(index=False, sep='\t'))